# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://www.reptune.tax")
ed.links

['https://informaconnect.com/tp-minds-international-conference/',
 'https://www.reptune.net/account/login',
 '/',
 '/full-services/',
 '/platform-detail/',
 '/service-plans/',
 '/price-list/',
 'https://reptune.tax/comparison/',
 '/about/',
 'https://share-eu1.hsforms.com/1katH0PdBTXWqXgJZ_01gCw2ekl1d',
 'https://share-eu1.hsforms.com/1TIIx5nqZRTiODIhkoLZgeg2ekl1d',
 '#',
 'https://www.reptune.tax/full-service/',
 'https://www.reptune.tax/software/',
 '#',
 'https://www.reptune.tax/full-service-pricing/',
 'https://www.reptune.tax/software-pricing/',
 'https://www.reptune.tax/comparison/',
 'https://www.reptune.tax/blogs/',
 'https://www.reptune.tax/about/',
 '#',
 'https://www.reptune.tax/full-service/',
 'https://www.reptune.tax/software/',
 '#',
 'https://www.reptune.tax/full-service-pricing/',
 'https://www.reptune.tax/software-pricing/',
 'https://www.reptune.tax/comparison/',
 'https://www.reptune.tax/blogs/',
 'https://www.reptune.tax/about/',
 'https://akerhorizons.com/',
 'htt

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.reptune.tax - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://informaconnect.com/tp-minds-international-conference/
https://www.reptune.net/account/login
/
/full-services/
/platform-detail/
/service-plans/
/price-list/
https://reptune.tax/comparison/
/about/
https://share-eu1.hsforms.com/1katH0PdBTXWqXgJZ_01gCw2ekl1d
https://share-eu1.hsforms.com/1TIIx5nqZRTiODIhkoLZgeg2ekl1d
#
https://www.reptune.tax/full-service/
https://www.reptune.tax/software/
#
https://www.reptune.tax/full-service-pricing/
https://www.reptune.tax/software-pricing/
https://www.reptune.tax/comparison/
https://www.reptune.tax/blogs/
https://www.reptune.tax/about/
#
https://www.reptune.tax/full-service/
https://www.reptune.tax/software/
#
https://www.reptune.tax/full-service-pri

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/#mainsa',
 '/#footer',
 'https://www.anthropic.com/',
 'https://www.anthropic.com/claude',
 'https://www.anthropic.com/team',
 'https://www.anthropic.com/enterprise',
 'https://www.anthropic.com/education',
 'https://www.anthropic.com/pricing',
 'https://claude.ai/download',
 'https://claude.ai/',
 'https://www.anthropic.com/news/claude-character',
 'https://www.anthropic.com/api',
 'https://docs.anthropic.com/',
 'https://www.anthropic.com/pricing#api',
 'https://console.anthropic.com/',
 'https://docs.anthropic.com/en/docs/welcome',
 'https://www.anthropic.com/solutions/agents',
 'https://www.anthropic.com/solutions/coding',
 'https://www.anthropic.com/solutions/customer-support',
 'https://www.anthropic.com/customers',
 'https://www.anthropic.com/research',
 'https://www.anthropic.com/economic-index',
 'https://www.anthropic.com/claude/sonnet',
 'https://www.anthropic.com/claude/haiku',
 'https://www.anthropic.com/news/claude-3-family',
 'https://www.anthropic.com/news/visible-ex

In [14]:
get_links("https://www.reptune.tax")

{'links': [{'type': 'about page', 'url': 'https://www.reptune.tax/about/'},
  {'type': 'full services page',
   'url': 'https://www.reptune.tax/full-services/'},
  {'type': 'price list page',
   'url': 'https://www.reptune.tax/full-service-pricing/'},
  {'type': 'software page', 'url': 'https://www.reptune.tax/software/'},
  {'type': 'comparison page', 'url': 'https://www.reptune.tax/comparison/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [15]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Team plan
Enterprise plan
Education plan
Explore pricing
Download apps
Claude log in
News
Claude’s character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collaborate with Claude
AI agents
Coding
Customer support
Case studies
Hear from our customers
Research
Research
Overview
Economic Index
Claude model family
Claude 3.7 Sonnet
Claude 3.5 Haiku
Claude 3 Opus
Research
Claude’s extended thinking
Commitments
Initiatives
Tr

In [28]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


'You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nChat with Claude\nOverview\nTeam plan\nEnterprise plan\nEducation plan\nExplore pricing\nDownload apps\nClaude log in\nNews\nClaude’s character\nAPI\nBuild with Claude\nAPI\xa0overview\nDeveloper docs\nExplore pricing\nConsole log in\nNews\nLearn how to build with Claude\nSolutions\nCollaborate with Claude\nAI\xa0agents\nCoding\nCustomer support\nCase studies\nHear from our customers\nResearch\nResearch\nOverview\nEconomic Index\nClaude model family\nClaude 3.7 Sonnet\nClaude 3.5 Haiku\nClaude 3 Opus\nResearch\nClaude’s extended thinking\nCommitments\nInitiatives\nTransparency\nResponsible scaling policy\nTrust center\nSecurity and compliance\nAnnouncement\nISO\xa042001 certification

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'education page', 'url': 'https://www.anthropic.com/education'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}, {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'}]}


# Anthropic Brochure

---

## Company Overview

**Anthropic** is an innovative AI safety and research company focused on building reliable, interpretable, and steerable AI systems. Headquartered in San Francisco, our mission is to ensure that the development of artificial intelligence serves humanity’s long-term well-being.

### Our Vision

At Anthropic, we believe that AI will greatly impact the world. We are dedicated to creating systems people can rely on and to conducting research that explores the opportunities and risks associated with AI. Our flagship product, Claude, exemplifies our commitment to safe AI by providing powerful tools for various applications.

---

## Our Commitment to Safety

We view AI safety as a systematic science. Our approach includes:

- **Research and Development**: Conducting frontier AI research across various modalities.
- **Interdisciplinary Collaboration**: Our diverse team of researchers, engineers, policy experts, and business leaders brings varied experiences to our work, enabling us to address complex challenges effectively.
- **Transparency and Communication**: We strive to communicate our findings to policymakers, industry stakeholders, and society to promote wider safety standards across the AI landscape.

---

## Company Culture

At Anthropic, our values are integral to how we operate and how we support our team:

1. **Act for the Global Good**: We aim to maximize positive outcomes for humanity while being bold in our actions.
2. **Hold Light and Shade**: We acknowledge the potential risks and benefits AI poses, working diligently to navigate them.
3. **Be Good to Our Users**: We define users broadly and prioritize kindness and generosity in all our interactions.
4. **Race to the Top on Safety**: We inspire a competitive dynamic to develop the safest AI systems in the market.
5. **Do the Simple Thing that Works**: Practicality and impact take precedence over complexity in problem-solving.
6. **Be Helpful, Honest, and Harmless**: We foster a high-trust culture, encouraging collaboration and open communication.
7. **Put the Mission First**: The mission guides our decisions, ensuring a shared purpose and collective ownership.

---

## Customer Base

Our products and services cater to a wide range of customers, including businesses, nonprofits, and educational institutions. By providing innovative AI solutions, we assist our clients in achieving efficiency and effectiveness in their operations.

---

## Career Opportunities

We are always looking for motivated individuals to join our team. With positions available across various disciplines—from research and engineering to operations—Anthropic fosters an inclusive and enriching workplace. 

### What We Offer:

- **Comprehensive Health Benefits**: Including dental, vision, and fertility support.
- **Flexible Paid Time Off**: 22 weeks of paid parental leave, plus ongoing wellness stipends.
- **Competitive Salaries and Equity Packages**: Along with retirement plans and relocation support.
- **Innovative Culture**: Collaborating with diverse teams to push the boundaries of AI.

### Application Process

Our technical interviews are designed to identify diverse talents with unique strengths. We welcome candidates from various backgrounds—even those without prior machine learning experience. 

If you are passionate about making AI safer and more beneficial for humanity, **join us** at Anthropic.

[Explore Open Roles](https://anthropic.com/careers)

---

## Get in Touch

If you want to learn more about our products and initiatives, visit our website [Claude.ai](https://claude.ai) or contact our sales team to discuss how we can assist your organization.

---

*Anthropic is a Public Benefit Corporation dedicated to responsible AI development for the long-term benefit of humanity.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [25]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        # response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [29]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'company page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'transparency page', 'url': 'https://www.anthropic.com/transparency'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'}]}


# Welcome to Anthropic: The Place Where AI Has To Behave!

*🦇 Your friendly neighborhood AI safety superheroes!*

---

## Who Are We? 🤖✨

At **Anthropic**, we don't just chat with machines; we make sure they don’t get too chatty and cause any chaos! Imagine a superhero team, but instead of capes, we wear lab coats, making AI systems that are reliable, interpretable, and safe for humanity. After all, what’s the point of having advanced AI if it can’t tell a good joke (or know why it’s not supposed to take over the world)?

## Meet Claude, Not a Guy in a Coat! 👔

Introducing **Claude**, our trusty AI companion who’s here to help you—without plotting to take over. Whether it’s **Claude 3.7 Sonnet**, **Claude 3.5 Haiku**, or even **Claude 3 Opus**, we assure you, they’re all about collaboration, coding, and the occasional serenade (only if you ask nicely).

## Our Values (Like Your Favorite Ingredients For Dalgona Coffee) ☕️❤️

1. **Act for the global good** - Because world domination is last season.
2. **Hold light and shade** - We believe in leveling up AI while holding the occasional existential crisis.
3. **Be good to our users** - Our clients and users are not just numbers; they’re more like delightful humans (or delightful non-humans if you’re an AI).
4. **Ignite a race to the top on safety** - No “last one to the finish line is a rotten egg” races here!
5. **Do the simple thing that works** - Let's not complicate things! No need for a spaceship if a bicycle will do!
6. **Be helpful, honest, and harmless** - We’re here to build trust, not snakes in the grass.
7. **Put the mission first** - Because at the end of the day, it's all about making tomorrow today’s smarter brother.

## A Peek into Our Culture 🏢🌱

Located in the bustling heart of **San Francisco**, our team is a diverse bunch—think of it as a buffet of skill sets with flavors like physics, machine learning, public policy, and the occasional fortune teller (just kidding). We’re a collaborative gang, spouting random AI facts and enjoying some plant neighbor cohabitation daily. 

### The Work-Life Balance (It's Not Just a Myth) 🌈

  - **Health & Wellness**
    - Comprehensive health plans to keep your body running smoother than an AI model!
    - Wellness stipends that will make you grin ear to ear—$500/month? Yes, please!

  - **Family Matters**
    - 22 weeks of paid parental leave—it's for all the fabulous parents out there embracing parenthood while keeping AI in check!

  - **Snacks & Meals** 
    - Daily meals and snacks to fuel your creative genius. (We really believe that the way to your best ideas is through your stomach.)

## Join Our Team! 🚀🦸‍♀️

Are you a clever cookie interested in keeping AI safe and helpful? We’re hiring! Dive into the adventurous world of AI research, engineering, and public policy with us. We want thinkers, builders, and even dreamers who want to mix with our band of tech aficionados.

### **Sampling the Interview Process** 📝

Worried about not having a background in machine learning? Don’t! At Anthropic, we’re diverse—you could’ve just blinked at a computer and learned Python! Our interview process is all about bringing your unique strengths to the table (or the virtual interview room). 

## Let’s Build a Future Together! 🌍🔮

If you think you have what it takes to help us on our quest to build safe AI, check out our open roles. The only thing we lack is someone to brainstorm a good name for our pet plant!

---

**Anthropic PBC**—Where Safety Meets Smarts!  
*Because AI should be your sidekick, not the tyrant.*

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>